In [1]:
import numpy as np
import pandas as pd
from PIL import Image
#import keras
#from keras.preprocessing import image

In [2]:
movies=pd.read_csv('archive/movies_metadata.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
movies_poster=movies['poster_path']

In [18]:
for i in range(1, movies_poster.size):
    Image.open(movies_poster[i])

FileNotFoundError: [Errno 2] No such file or directory: '/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg'